### Today, we will scrape Producthunt with Selenium

Selenium is a tool initially created to automate tests on websites. It is therefore very useful when information is accessible by clicking on links. A button for example is an element on which it is very difficult to obtain the link. Beautifull soup then becomes limited.
In this case, use Selenium

### Load libraries

In [3]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os
date=strftime("%Y-%m-%d")


### Install Selenium according to this manual

https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium/bin

nb: Linux: put your geckodriver (the downloaded extension) in the equivalent path at home to /home/YOURNAME/.local/bin

We will simulate a search on the official Phython website

In [11]:
import selenium

# The selenium.webdriver module provides all the implementations of WebDriver. WebDriver implementations 
# Currently supported are Firefox, Chrome, IE and Remote. The Keys class provides keys in 
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# Then, the instance of Firefox WebDriver is created.
driver = webdriver.Chrome()
# The driver.get method will lead to a page given by the URL. WebDriver will wait until the page is fully completed 
# loaded (i.e. the "onload" event has been triggered) before returning the control to your script. 
# It should be noted that if your page uses a lot of AJAX when loading, WebDriver may not know 
# when it was fully charged:
driver.get("http://www.python.org")
# The following line is a statement confirming that the title contains the word "Python"
assert "Python" in driver.title
# WebDriver offers several methods to search for items using one of the methods 
# find_element_by_by_ * . For example, the input text element can be located by its name attribute by 
# using the find_element_by_name method 
elem = driver.find_element_by_name("q")
# Then we send keys. This is similar to entering keys using your keyboard. 
# Special keys can be sent using the imported selenium.webdriver.common.keys Keys class. 
# For security reasons, we will delete any pre-filled text in the input field 
# (for example, "Search") so that it does not affect our search results:


elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)

#After submitting the page, you should get the result if there is one. To ensure that certain results 
# are found, make an assertion:
assert "No results found." not in driver.page_source
driver.close()

#### Open the source code of this page (HTML) and check that the search area (field) is called "q".

input id="id-search-field" name="q" type="search" role="textbox" class="search-field placeholder" placeholder="Search" value="" tabindex="1"

In [15]:
url='https://www.python.org/'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
for p in soup.find_all('input'):
    print (p.get("name"))

https://www.python.org/ 200
q


### Getting a phone number from an ad in the right place

In [25]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

url='https://www.leboncoin.fr/equipement_auto/1906151573.htm'

driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(url)
time.sleep(2)

python_button = driver.find_element(By.CSS_SELECTOR,'[title="voir le numéro"]')
time.sleep(10)
python_button.click()

# And then it's like Beautiful soup
soup=BeautifulSoup(driver.page_source)

driver.close()
telephone=""

# And then it's like Beautiful soup
for elem in soup.find_all('a',attrs={"title":"appeler"}):
    telephone = elem.text
    print(elem.text)

06 13 01 24 29


### Starting from an ad in the right place, collect all the information available to define the product being sold. Use selenium for the telephone number

In [36]:
for elem in soup.find_all("h1", attrs={"data-qa-id":"adview_title"}):
    print("Title: "+elem.text, "\n")

    
for elem in soup.find_all("div", attrs={"data-qa-id":"adview_price"})[0]:
    print("Prix: "+elem.next_element.text, "\n")

for elem in soup.find_all("p", attrs={"data-qa-id":"adview_date"})[0]:
    print("Ajouter le: "+elem, "\n")

for elem in soup.find_all("span", attrs={"class":"_1fFkI"}):
    print("Description: "+elem.text, "\n")


Title: Pneus Occasions et Neufs R15 

Prix: 25 € 

Ajouter le: 16/04/2021 à 11:10 

Description: Docteur Pneus vous propose des pneus de toutes marques Michelin, Goodyear Bridgestone, Hankook, Continental... à partir de 25€ le pneu.
Et comme nous aimons autant prendre soin de vous que de votre auto, nous vous offrons le montage et l’équilibrage :o)

Exemple :
185/65/15 : 50€ la paire 
195/65/15 : 50€ la paire 
185/60/15 : 50€ la paire 

Nous avons toutes les dimensions : 
205/65/15 195/55/15 185/55/15 195/50/15 205/60/15 195/60/15 165/60/15 165/65/15 205/70/15 mais également des pneus camio... 



### API (Application Program Interface)

A set of tools and methods that allow different applications to interact with each other. In the case of a web service, we can retrieve data dynamically. By using an API correctly, we can thus obtain in real time, the modifications made on a "parent" site.

For example, we will retrieve online news, for example from the "L'équipe" website.

Follow the instructions at https://newsapi.org/s/lequipe-api to retrieve an "API key" connection key

Your API key is: 73bbb95f8ecb49b499113a46481b4af1


It is frequent that a key does not work after a while 5mins 30 mins 1 d ...
So don't jump up if you get an error message back.

In [37]:
import requests
key='73bbb95f8ecb49b499113a46481b4af1'
url ='https://newsapi.org/v2/top-headlines?sources=lequipe&apiKey='+key
response = requests.get(url)

# Here the response format is a json file, it is used as a dictionary

print (response.json())

{'status': 'ok', 'totalResults': 10, 'articles': [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Les plus beaux buts redessinés #7\xa0: Ilan, un ciseau de haute voltige (PSG - Saint-Étienne 2007)', 'description': "Le 25\xa0février, au Parc des Princes, l'attaquant stéphanois concrétise <i>«\xa0un pari un peu fou\xa0»</i> avec son agent en marquant\xa0le plus beau but de la saison en L1. Décryptage animé et commenté.", 'url': 'https://www.lequipe.fr/Football/Actualites/Les-plus-beaux-buts-redessines-7-ilan-un-ciseau-de-haute-voltige-psg-saint-etienne-2007/1242507', 'urlToImage': 'https://medias.lequipe.fr/img-video-cover/1500000001476233/640/0', 'publishedAt': '2021-04-16T10:00:00+00:00', 'content': "Tout est parti d'un pari avec son agent. Ilan avait écrit le scénario d'avance. Ce 25\xa0février 2007, il était dit que l'attaquant brésilien signerait un bijou. Le plus brillant, le mieux ciselé de la … [+270 chars]"}, {'source': {'id': 'lequipe', 'name':

In [38]:
dic=response.json()
print(dic.keys())

dict_keys(['status', 'totalResults', 'articles'])


In [39]:
for elem in list(dic.keys()):
    print('##############################################')
    print("clé: ",elem,"// values: ", dic[elem])

##############################################
clé:  status // values:  ok
##############################################
clé:  totalResults // values:  10
##############################################
clé:  articles // values:  [{'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Les plus beaux buts redessinés #7\xa0: Ilan, un ciseau de haute voltige (PSG - Saint-Étienne 2007)', 'description': "Le 25\xa0février, au Parc des Princes, l'attaquant stéphanois concrétise <i>«\xa0un pari un peu fou\xa0»</i> avec son agent en marquant\xa0le plus beau but de la saison en L1. Décryptage animé et commenté.", 'url': 'https://www.lequipe.fr/Football/Actualites/Les-plus-beaux-buts-redessines-7-ilan-un-ciseau-de-haute-voltige-psg-saint-etienne-2007/1242507', 'urlToImage': 'https://medias.lequipe.fr/img-video-cover/1500000001476233/640/0', 'publishedAt': '2021-04-16T10:00:00+00:00', 'content': "Tout est parti d'un pari avec son agent. Ilan avait écrit le scénario d'ava

In [40]:
# And now we have lists in dictionaries it's json but it's always the same thing
# We will discover the information of the article key
for elem in enumerate(dic['articles']):
    print('###############################################')
    print(elem)

###############################################
(0, {'source': {'id': 'lequipe', 'name': "L'equipe"}, 'author': "L'EQUIPE", 'title': 'Les plus beaux buts redessinés #7\xa0: Ilan, un ciseau de haute voltige (PSG - Saint-Étienne 2007)', 'description': "Le 25\xa0février, au Parc des Princes, l'attaquant stéphanois concrétise <i>«\xa0un pari un peu fou\xa0»</i> avec son agent en marquant\xa0le plus beau but de la saison en L1. Décryptage animé et commenté.", 'url': 'https://www.lequipe.fr/Football/Actualites/Les-plus-beaux-buts-redessines-7-ilan-un-ciseau-de-haute-voltige-psg-saint-etienne-2007/1242507', 'urlToImage': 'https://medias.lequipe.fr/img-video-cover/1500000001476233/640/0', 'publishedAt': '2021-04-16T10:00:00+00:00', 'content': "Tout est parti d'un pari avec son agent. Ilan avait écrit le scénario d'avance. Ce 25\xa0février 2007, il était dit que l'attaquant brésilien signerait un bijou. Le plus brillant, le mieux ciselé de la … [+270 chars]"})
##################################

In [41]:
# So if we keep going, it gives us another dictionary!
for elem in dic['articles'][0].keys():
    print(' Key : ',elem,'Values : ',dic['articles'][0][elem])

 Key :  source Values :  {'id': 'lequipe', 'name': "L'equipe"}
 Key :  author Values :  L'EQUIPE
 Key :  title Values :  Les plus beaux buts redessinés #7 : Ilan, un ciseau de haute voltige (PSG - Saint-Étienne 2007)
 Key :  description Values :  Le 25 février, au Parc des Princes, l'attaquant stéphanois concrétise <i>« un pari un peu fou »</i> avec son agent en marquant le plus beau but de la saison en L1. Décryptage animé et commenté.
 Key :  url Values :  https://www.lequipe.fr/Football/Actualites/Les-plus-beaux-buts-redessines-7-ilan-un-ciseau-de-haute-voltige-psg-saint-etienne-2007/1242507
 Key :  urlToImage Values :  https://medias.lequipe.fr/img-video-cover/1500000001476233/640/0
 Key :  publishedAt Values :  2021-04-16T10:00:00+00:00
 Key :  content Values :  Tout est parti d'un pari avec son agent. Ilan avait écrit le scénario d'avance. Ce 25 février 2007, il était dit que l'attaquant brésilien signerait un bijou. Le plus brillant, le mieux ciselé de la … [+270 chars]


### Make a script that allows you to take details of the last ten news from the team or another site. Store them in a nice csv or excel file

In [44]:
for elem in dic['articles'][:10]:
    print(elem["content"])
    print("##################################")

Tout est parti d'un pari avec son agent. Ilan avait écrit le scénario d'avance. Ce 25 février 2007, il était dit que l'attaquant brésilien signerait un bijou. Le plus brillant, le mieux ciselé de la … [+270 chars]
##################################
Auteur de 25 buts en 35 matches toutes compétitions confondues cette saison, Karim Benzema est plus que jamais le leader d'attaque du Real Madrid. Ses performances autant en Liga qu'en Ligue des cham… [+660 chars]
##################################
La Copa America se jouera « avec ou sans public », qu'importe l'évolution du Covid-19 en Amérique du Sud, a jugé jeudi le ministre colombien des Sports, après des propos moins optimistes du président… [+1195 chars]
##################################
La piste de dix-neuf virages et de 3,32 kilomètres verra les voitures monter la colline après le premier virage à Sainte Dévote. Elle comprendra des virages emblématiques, notamment la place du Casin… [+73 chars]
##################################
Comm